This is a starter notebook for the project, you'll have to import the libraries you'll need, you can find a list of the ones available in this workspace in the requirements.txt file in this workspace. 

In [ ]:
# !pip install pydantic>=1.10.12
# !pip install pytest>=7.4.0
# !pip install sentence-transformers>=2.2.0
# !pip install transformers>=4.31.0
# !pip install chromadb==0.4.18
# !pip install --upgrade --quiet  langchain-openai
# !pip install --upgrade openai
!pip install langchain==0.0.316 --user
!pip install openai==0.28.1 --user
# !pip install -U langchain chromadb
# !pip install chromadb==0.4.15
# !pip install openai==1.12.0
!pip install langchain==0.0.316
!pip install openai==0.28.1
# !pip install langchain-openai --user
# !pip install langchain-community langchain-core
# !pip install --upgrade langchain
# !pip install --upgrade pip --user
# !pip install langchain --upgrade --user
# !pip install --upgrade openai --user


[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
import io
import os
import sys
import shutil
# Import Azure OpenAI
import openai
import numpy as np
import pandas as pd
import io


In [17]:
from langchain.prompts import PromptTemplate
from chromadb.utils import embedding_functions
from chromadb import Documents, EmbeddingFunction, Embeddings
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_openai import AzureOpenAI, AzureOpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain

from langchain.prompts import ChatPromptTemplate
from langchain_openai import AzureChatOpenAI

In [2]:
"Pandoras-Box"
"Enigma-code"
"https://enigma-code.openai.azure.com"
"ea5e645a87f04a72beed5bdffde87da2"

'ea5e645a87f04a72beed5bdffde87da2'

### Generating Real Estate Listings with an LLM
- using a Large Language Model (LLM) to generate at least 10 diverse and realistic real estate listings containing facts about the real estate.

In [18]:

prompt='''
Create a comprehensive CSV file encapsulating the essence of {num_listings} distinct real estate listings. Each listing should be organized into columns, showcasing the following attributes:

- Neighborhood: Identify the neighborhood where the property resides, such as "Green Oaks."
- Price: State the property's market price in USD, formatted (e.g., "$800,000").
- Bedrooms: Enumerate the bedrooms within the property (e.g., 3).
- Bathrooms: Count the property's bathrooms (e.g., 2).
- House Size: Detail the property's square footage (e.g., "2,000 sqft").

Elevate each property's presentation with a comprehensive paragraph that accentuates its unique features, amenities, and sustainable qualities. Highlight elements like energy-efficient appliances, solar panel integration, use of sustainable materials, and the presence of gardens.

Example Entry Format:
Neighborhood,Price,Bedrooms,Bathrooms,House Size,Description
Green Oaks,"$800,000",3,2,"2,000 sqft","Nestled in Green Oaks, this eco-friendly haven features a 3-bedroom, 2-bathroom layout with solar panels and efficient insulation. Highlights include abundant natural light, hardwood floors, and an open-concept kitchen that leads to a lush backyard, embodying a sanctuary for eco-conscious living. The neighborhood of Green Oaks is celebrated for its vibrant and environmentally-aware community, boasting organic stores, community gardens, and convenient transit options, rendering it perfect for those prioritizing sustainability and community engagement."


Ensure the description not only reflects the property's allure, such as its eco-friendly design and comfortable living spaces but also paints a vivid picture of the neighborhood's character. Emphasize community elements like Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze.

Structure the CSV with clear headers for each column. Follow the example provided to format each subsequent row with information specific to a different property listing.
Make sure you generate {num_listings} distinct and unique listings.
'''

openai.api_type = "azure"
openai.api_version = "2023-03-15-preview" 
openai.api_base =  "https://enigma-code.openai.azure.com"  # Your Azure OpenAI resource's endpoint value.
openai.api_key = "ea5e645a87f04a72beed5bdffde87da2"

gen_listing_prompt = PromptTemplate.from_template(prompt).format(num_listings=15)

conversation=[{"role": "system", "content": f"{gen_listing_prompt}"}]

response = openai.ChatCompletion.create(
    engine="Enigma-code", # The deployment name you chose when you deployed the GPT-3.5-Turbo or GPT-4 model.
    messages = conversation,
    temperature=0,
    max_tokens=2400
)
agent_response = response.choices[0].message.content
conversation.append({"role": "assistant", "content": agent_response})
print(agent_response)



Neighborhood,Price,Bedrooms,Bathrooms,House Size,Description
Westwood,"$1,200,000",4,3,"2,500 sqft","This stunning 4-bedroom, 3-bathroom home in Westwood boasts a spacious open floor plan, perfect for entertaining. The property features a chef's kitchen with stainless steel appliances, a large master suite with a walk-in closet, and a backyard oasis with a pool and spa. The neighborhood of Westwood is known for its excellent schools, tree-lined streets, and proximity to trendy restaurants and shops."
Brentwood,"$2,500,000",5,4,"3,800 sqft","Nestled in the heart of Brentwood, this luxurious 5-bedroom, 4-bathroom home boasts high-end finishes and an open-concept floor plan. The property features a gourmet kitchen with top-of-the-line appliances, a spacious master suite with a fireplace and balcony, and a backyard with a built-in BBQ and fire pit. The neighborhood of Brentwood is known for its upscale shops and restaurants, excellent schools, and easy access to the beach."
Santa Monica,"$

In [19]:
type(agent_response)

str

In [22]:
print("Datatype of the agent response: ", type(agent_response), "\n")
# Transform our listings to dataframe
homes_df = pd.read_csv(io.StringIO(agent_response), sep=",")
display(homes_df)

# Saving the Homes listings in a csv file
homes_df.to_csv('Homes_listings.csv', index=None)

Datatype of the agent response:  <class 'str'> 



,Neighborhood,Price,Bedrooms,Bathrooms,House Size,Description
0,Westwood,"$1,200,000",4,3,"2,500 sqft","This stunning 4-bedroom, 3-bathroom home in We..."
1,Brentwood,"$2,500,000",5,4,"3,800 sqft","Nestled in the heart of Brentwood, this luxuri..."
2,Santa Monica,"$1,800,000",3,2,"1,800 sqft","This charming 3-bedroom, 2-bathroom home in Sa..."
3,Venice,"$1,500,000",2,2,"1,500 sqft","Located in the heart of Venice, this stunning ..."
4,Marina Del Rey,"$1,700,000",3,3,"2,200 sqft","This beautiful 3-bedroom, 3-bathroom home in M..."
5,Culver City,"$1,000,000",2,2,"1,200 sqft","This charming 2-bedroom, 2-bathroom home in Cu..."
6,Playa Vista,"$1,600,000",4,3,"2,500 sqft","Located in the heart of Playa Vista, this stun..."
7,Westchester,"$900,000",3,2,"1,500 sqft","This charming 3-bedroom, 2-bathroom home in We..."
8,Pacific Palisades,"$3,000,000",5,4,"4,000 sqft","Nestled in the hills of Pacific Palisades, thi..."
9,Beverly Hills,"$5,000,000",6,5,"5,500 sqft","This stunning 6-bedroom, 5-bathroom home in Be..."


### Creating a Vector database and Storing Listings

In [3]:
"""
os.environ["OPENAI_API_VERSION"] = '2023-03-15-preview'
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://enigma-code.openai.azure.com"
os.environ["AZURE_OPENAI_API_KEY"] = "ea5e645a87f04a72beed5bdffde87da2"

loader = CSVLoader(file_path='Homes_listings.csv')
docs = loader.load()

# print(docs)
llm = AzureOpenAI(deployment_name='Enigma-code', temperature=0, max_tokens = 2000)

splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
split_docs = splitter.split_documents(docs)

embeddings = AzureOpenAIEmbeddings(
    azure_deployment="Pandoras-Box",
    openai_api_version="2023-03-15-preview",
)
db = Chroma.from_documents(split_docs, embeddings)
query = """
#    Would like to buy home with 3 bedrooms

"""
use_chain_helper = False
if use_chain_helper:
    rag = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=db.as_retriever())
    print(rag.run(query))
else:
    similar_docs = db.similarity_search(query, k=5)
    prompt = PromptTemplate(
        template="{query}\nContext: {context}",
        input_variables=["query", "context"],
    )
    chain = load_qa_chain(llm, prompt = prompt, chain_type="stuff")
    print(chain.run(input_documents=similar_docs, query = query))
"""

'\nuse_chain_helper = False\nif use_chain_helper:\n    rag = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=db.as_retriever())\n    print(rag.run(query))\nelse:\n    similar_docs = db.similarity_search(query, k=5)\n    prompt = PromptTemplate(\n        template="{query}\nContext: {context}",\n        input_variables=["query", "context"],\n    )\n    chain = load_qa_chain(llm, prompt = prompt, chain_type="stuff")\n    print(chain.run(input_documents=similar_docs, query = query))\n'

In [23]:
import pandas as pd

homes_df = pd.read_csv('Homes_listings.csv')
homes_df.head()

,Neighborhood,Price,Bedrooms,Bathrooms,House Size,Description
0,Westwood,"$1,200,000",4,3,"2,500 sqft","This stunning 4-bedroom, 3-bathroom home in We..."
1,Brentwood,"$2,500,000",5,4,"3,800 sqft","Nestled in the heart of Brentwood, this luxuri..."
2,Santa Monica,"$1,800,000",3,2,"1,800 sqft","This charming 3-bedroom, 2-bathroom home in Sa..."
3,Venice,"$1,500,000",2,2,"1,500 sqft","Located in the heart of Venice, this stunning ..."
4,Marina Del Rey,"$1,700,000",3,3,"2,200 sqft","This beautiful 3-bedroom, 3-bathroom home in M..."


In [6]:
import openai
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain import LLMChain
from langchain.chains.question_answering import load_qa_chain

from langchain_openai import AzureOpenAI
from langchain_openai import AzureOpenAIEmbeddings

In [25]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_openai import AzureOpenAI

from langchain_openai import AzureOpenAIEmbeddings
from langchain.schema import Document

In [29]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain import LLMChain
from langchain.chains.question_answering import load_qa_chain

from langchain_openai import AzureOpenAI
from langchain_openai import AzureOpenAIEmbeddings

In [38]:
# !pip install langchain==0.0.316 --user
# !pip install openai==0.28.1 --user
# !pip install langchain openai --upgrade



In [33]:
CHROMA_PATH = "chroma"
if chunks:
    document = chunks[10]
    print(document.page_content)
    print(document.metadata)

# Save to Chroma
if os.path.exists(CHROMA_PATH):
    shutil.rmtree(CHROMA_PATH)

db = Chroma.from_documents(
    chunks, AzureOpenAIEmbeddings(input = chunks, engine="Pandoras-Box"), persist_directory=CHROMA_PATH
)
db.persist()
print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")

Located in the heart of Venice, this stunning 2-bedroom, 2-bathroom home features an open-concept living space with high ceilings and skylights, a gourmet kitchen with stainless steel appliances, and
{'id': '3', 'start_index': 0}


C:\ProgramData\Anaconda3\lib\site-packages\langchain_openai\embeddings\base.py:148: UserWarning: WARNING! input is not default parameter.
                    input was transferred to model_kwargs.
                    Please confirm that input is what you intended.
  warnings.warn(


ValidationError: 1 validation error for AzureOpenAIEmbeddings
__root__
  As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`).  (type=value_error)

In [30]:

import os
import pandas as pd
import shutil


from langchain.evaluation import load_evaluator
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.chroma import Chroma
from dataclasses import dataclass
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

ModuleNotFoundError: No module named 'langchain.evaluation'

In [44]:



docs = []
for index, row in homes_df.iterrows():
    docs.append(Document(page_content=row['Description'], metadata={'id': str(index)}))
    
# Split Text
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=100,
    length_function=len,
    add_start_index=True,
)
chunks = text_splitter.split_documents(docs)
print(f"Split {len(docs)} documents into {len(chunks)} chunks.")

os.environ["OPENAI_API_VERSION"] = '2023-03-15-preview'
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://enigma-code.openai.azure.com"
os.environ["AZURE_OPENAI_API_KEY"] = "ea5e645a87f04a72beed5bdffde87da2"



if chunks:
    document = chunks[10]
    print(document.page_content)
    print(document.metadata)
    
import openai

openai.api_type = "azure"
openai.api_key = "ea5e645a87f04a72beed5bdffde87da2"
openai.api_base = "https://enigma-code.openai.azure.com"
openai.api_version = "2023-03-15-preview"

response = openai.Embedding.create(
    input = document.page_content,
    engine="Pandoras-Box"
)
embeddings = response['data'][0]['embedding']
# print(embeddings)
  
    
db = Chroma.from_documents(chunks, embeddings , persist_directory= "chroma")
db.persist()

Split 15 documents into 55 chunks.
Located in the heart of Venice, this stunning 2-bedroom, 2-bathroom home features an open-concept living space with high ceilings and skylights, a gourmet kitchen with stainless steel appliances, and
{'id': '3', 'start_index': 0}


AttributeError: 'list' object has no attribute 'embed_documents'

In [8]:

# set the environment variables needed for openai package to know to reach out to azure
import os



os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["azure_endpoint"] = "https://enigma-code.openai.azure.com"
os.environ["OPENAI_API_KEY"] = "ea5e645a87f04a72beed5bdffde87da2"
os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(engine="Pandoras-Box", chunk_size=1)

text = "This is a test document."

query_result = embeddings.embed_query(text)

doc_result = embeddings.embed_documents([text])




ValidationError: 1 validation error for OpenAIEmbeddings
__root__
  If you are using Azure, please use the `AzureOpenAIEmbeddings` class. (type=value_error)

In [37]:
import openai

openai.api_type = "azure"
openai.api_key = "ea5e645a87f04a72beed5bdffde87da2"
openai.api_base = "https://enigma-code.openai.azure.com"
openai.api_version = "2023-03-15-preview"

response = openai.Embedding.create(
    input = document.page_content,
    engine="Pandoras-Box"
)
embeddings = response['data'][0]['embedding']
print(embeddings)

[-0.0021057503763586283, 0.00725912069901824, 0.01883738487958908, 0.0038717552088201046, 0.007797331549227238, 0.022604860365390778, -0.012984340079128742, -0.015473566018044949, 0.0064518041908741, -0.024138761684298515, 0.013663831166923046, 0.025847582146525383, -0.017424579709768295, -0.021326610818505287, -0.0023344901856034994, -0.0018938299035653472, 0.01618669554591179, 0.010629666969180107, 0.024865347892045975, -0.026897093281149864, -0.03823988884687424, 0.005372018553316593, 0.007252392824739218, -0.008322087116539478, -0.0013547779526561499, -0.008772838860750198, 0.023344900459051132, -0.021501528099179268, -0.017276572063565254, 0.007360035087913275, 0.015002631582319736, -0.014289501123130322, -0.010279829613864422, -0.017222750931978226, -0.005459477659314871, -0.0006656156037934124, -0.011881007812917233, 0.0002478713868185878, 0.013791656121611595, -0.012991067953407764, 0.020707666873931885, -0.009324504993855953, 0.011295703239738941, 0.001443919143639505, -0.0047

### Semantic Search of Listings Based on Buyer Preferences
- listings are semantically searched based on given buyer preferences. The search should return listings that closely match the input preferences.

In [45]:

PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Given the context provided above, create a response that not only answers the question {question}, with

ensures that your explanation is distinct, captivating, and customized to align with the specified preferences. Strive to present your insights in a manner that resonates with the audience's interests and requirements
"""
query_text = "Would like to buy home in calm neighbourhood" 

# Prepare the DB.


embeddings = AzureOpenAIEmbeddings(
    azure_deployment="Pandoras-Box",
    openai_api_version="2023-03-15-preview",
)
db = Chroma(persist_directory = "chroma", embedding_function=embeddings)

# Search the DB.
results = db.similarity_search_with_relevance_scores(query_text, k=3)
if len(results) == 0 or results[0][1] < 0.7:
    print(f"Unable to find matching results.")
else:
    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)
    print(f"Generated Prompt:\n{prompt}")

    model = AzureChatOpenAI(
    openai_api_version="2023-03-15-preview",
    azure_deployment="Enigma-code",
)
    response_text = model.predict(prompt)

    sources = [doc.metadata.get("id", None) for doc, _score in results]
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    print(formatted_response)

ValidationError: 1 validation error for AzureOpenAIEmbeddings
__root__
  As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`).  (type=value_error)

In [34]:
# !pip install "openai==0.28.1" num2words matplotlib plotly scipy scikit-learn pandas tiktoken


In [ ]:
import openai
import os
import re
import requests
import sys
from num2words import num2words
import os
import pandas as pd
import numpy as np
from openai.embeddings_utils import get_embedding, cosine_similarity
import tiktoken

API_KEY = os.getenv("AZURE_OPENAI_API_KEY") 
RESOURCE_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT") 

openai.api_type = "azure"
openai.api_key = API_KEY
openai.api_base = RESOURCE_ENDPOINT
openai.api_version = "2022-12-01"

url = openai.api_base + "/openai/deployments?api-version=2022-12-01" 

r = requests.get(url, headers={"api-key": API_KEY})

print(r.text)